In [4]:
import requests
import pandas as pd
from datetime import datetime

In [5]:
def get_mortgage_loan_products(api_key, merge_df=False):
    """
    금융감독원 주택담보대출 상품 정보를 가져와서 DataFrame으로 변환하는 함수

    Parameters:
        api_key (str): 금융감독원 API 인증키
        merge_df (bool): DataFrame 병합 여부 (True: 단일 DataFrame 반환, False: 기본/옵션 정보 별도 반환)

    Returns:
        DataFrame 또는 tuple: merge_df 값에 따라 단일 DataFrame 또는 (기본정보 DataFrame, 옵션정보 DataFrame) 반환
    """

    base_url = "http://finlife.fss.or.kr/finlifeapi/mortgageLoanProductsSearch.json"
    params = {
        "auth": api_key,
        "topFinGrpNo": "020000",
        "pageNo": "1"
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        base_list = data.get('result', {}).get('baseList', [])
        option_list = data.get('result', {}).get('optionList', [])

        df_base = pd.DataFrame(base_list)
        df_option = pd.DataFrame(option_list)

        # 기본 정보 컬럼명 한글로 변경
        base_column_mapping = {
            'fin_co_no': '금융회사코드',
            'kor_co_nm': '금융회사명',
            'fin_prdt_cd': '금융상품코드',
            'fin_prdt_nm': '금융상품명',
            'join_way': '가입방법',
            'loan_inci_expn': '대출부대비용',
            'erly_rpay_fee': '중도상환수수료',
            'dly_rate': '연체이자율',
            'loan_lmt': '대출한도',
            'dcls_strt_day': '공시시작일',
            'dcls_end_day': '공시종료일'
        }

        # 옵션 정보 컬럼명 한글로 변경
        option_column_mapping = {
            'fin_co_no': '금융회사코드',
            'fin_prdt_cd': '금융상품코드',
            'mrtg_type': '담보유형코드',
            'mrtg_type_nm': '담보유형',
            'rpay_type': '대출상환유형코드',
            'rpay_type_nm': '대출상환유형',
            'lend_rate_type': '금리구분코드',
            'lend_rate_type_nm': '금리구분',
            'lend_rate_min': '금리_최저',
            'lend_rate_max': '금리_최고',
            'lend_rate_avg': '금리_평균'
        }

        df_base = df_base.rename(columns=base_column_mapping)
        df_option = df_option.rename(columns=option_column_mapping)

        # 날짜 데이터 처리
        df_base['공시시작일'] = pd.to_datetime(df_base['공시시작일'], format='%Y%m%d', errors='coerce')
        df_base['공시종료일'] = pd.to_datetime(df_base['공시종료일'], format='%Y%m%d', errors='coerce')

        # 현재 유효한 상품 표시
        current_date = pd.Timestamp.now()
        df_base['현재유효여부'] = (
            (df_base['공시시작일'] <= current_date) &
            ((df_base['공시종료일'].isna()) | (df_base['공시종료일'] >= current_date))
        )

        # 숫자형 데이터 처리
        numeric_columns = ['금리_최저', '금리_최고', '금리_평균']
        df_option[numeric_columns] = df_option[numeric_columns].apply(pd.to_numeric, errors='coerce')

        if merge_df:
            # DataFrame 병합
            df_merged = pd.merge(
                df_base,
                df_option,
                on=['금융회사코드', '금융상품코드'],
                how='left'
            )

            # 컬럼 순서 재정렬
            column_order = [
                '금융회사명', '금융상품명', '담보유형', '대출상환유형',
                '금리구분', '금리_최저', '금리_최고', '금리_평균',
                '대출한도', '대출부대비용', '중도상환수수료', '연체이자율',
                '가입방법', '공시시작일', '공시종료일', '현재유효여부',
                '금융회사코드', '금융상품코드', '담보유형코드', '대출상환유형코드', '금리구분코드'
            ]

            # 존재하는 컬럼만 선택
            final_columns = [col for col in column_order if col in df_merged.columns]
            df_merged = df_merged[final_columns]

            return df_merged

        return df_base, df_option

    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"데이터 처리 중 오류 발생: {e}")
        return None

In [6]:
API_KEY = ""  # 발급받은 API 키를 입력하세요

    # 단일 DataFrame으로 받기
df = get_mortgage_loan_products(API_KEY, merge_df=True)

if df is not None:
    print("데이터 수집 완료")
    print(f"총 상품 수: {len(df)}")
    print("\n=== 데이터 미리보기 ===")
    print(df[['금융회사명', '금융상품명', '담보유형', '금리_최저', '금리_최고', '현재유효여부']].head())

    # 데이터 저장
    # df.to_csv('mortgage_loan_full.csv', index=False, encoding='utf-8-sig')

데이터 수집 완료
총 상품 수: 120

=== 데이터 미리보기 ===
  금융회사명   금융상품명  담보유형  금리_최저  금리_최고  현재유효여부
0  우리은행  우리아파트론   아파트   4.46   5.66    True
1  우리은행  우리아파트론   아파트   5.26   6.46    True
2  우리은행  우리아파트론   아파트   5.06   6.16    True
3  우리은행  우리부동산론  아파트외   4.66   5.86    True
4  우리은행  우리부동산론  아파트외   5.26   6.46    True


In [ ]:
# 사용 예시:
"""
# 1. 기본정보와 옵션정보를 별도로 받기
df_base, df_option = get_mortgage_loan_products(API_KEY, merge_df=False)

# 2. 하나의 DataFrame으로 받기
df = get_mortgage_loan_products(API_KEY, merge_df=True)

# 3. 데이터 분석 예시
# 금융회사별 평균 금리 확인
print(df.groupby('금융회사명')[['금리_최저', '금리_최고', '금리_평균']].mean())

# 담보유형별 상품 수 확인
print(df.groupby('담보유형')['금융상품명'].count())

# 현재 유효한 상품만 필터링
active_products = df[df['현재유효여부']]
"""